## Installing Packages 

In [2]:
%pip install pandas matplotlib seaborn

     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
     - -------------------------------------- 0.4/10.7 MB 12.6 MB/s eta 0:00:01
     ---- ----------------------------------- 1.2/10.7 MB 12.3 MB/s eta 0:00:01
     ------ --------------------------------- 1.8/10.7 MB 12.8 MB/s eta 0:00:01
     ----------- ---------------------------- 3.1/10.7 MB 16.7 MB/s eta 0:00:01
     ------------------ --------------------- 4.9/10.7 MB 19.4 MB/s eta 0:00:01
     ------------------------ --------------- 6.6/10.7 MB 22.3 MB/s eta 0:00:01
     ----------------------------------- ---- 9.4/10.7 MB 27.2 MB/s eta 0:00:01
     ------------------------------------- - 10.3/10.7 MB 26.2 MB/s eta 0:00:01
     --------------------------------------  10.6/10.7 MB 25.1 MB/s eta 0:00:01
     --------------------------------------  10.7/10.7 MB 23.4 MB/s eta 0:00:01
     --------------------------------------  10.7/10.7 MB 23.4 MB/s eta 0:00:01
     --------------------------------------  10.


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Packages

In [3]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns